# Mapping Census Data (Work in Progress)

By Kenneth Burchfiel

Released under the MIT License

This notebook will demonstrate how to create choropleth maps of data retrieved via census_data_imports.ipynb. Specifically, it will examine and visualize correlations between marital status and poverty rates within the American Community Survey.

**Note: See choropleth_maps.ipynb for additional documentation.**

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
import folium
import numpy as np
import os
import geopandas
from choropleth_map_functions import cptt, create_map_and_screenshot

display_maps = False 
skipped_render_explanation = "Skipping map output in order \
to allow this notebook to display on GitHub. Set display_maps to True \
in order to show this and other maps within the notebook."


# Part 1: Creating a data source for our visualizations

Reading in Census data created within the Census Data Imports section of Python for Nonprofits:

In [2]:
df_marriage_poverty_census_data = pd.read_csv(
    '../Census_Data_Imports/Datasets/marriage_poverty_census_data.csv')
# Renaming the NAME column in order to avoid a conflict with the NAME column
# within our shapefile dataset:
df_marriage_poverty_census_data.rename(
    columns = {'NAME':'County/State'},
    inplace = True)
df_marriage_poverty_census_data.head()

,County/State,Sex by Age_Estimate!!Total: (B01001_001E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total: (B17010_001E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Married-couple family: (B17010_003E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Married-couple family:!!With related children of the householder under 18 years: (B17010_004E),"Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Male householder, no spouse present:!!With related children of the householder under 18 years: (B17010_011E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Female householder, no spouse present: (B17010_016E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Female householder, no spouse present:!!With related children of the householder under 18 years: (B17010_017E)",Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Married-couple family: (B17010_023E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Married-couple family:!!With related children of the householder under 18 years: (B17010_024E),"Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Male householder, no spouse present:!!With related children of the householder under 18 years: (B17010_031E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Female householder, no spouse present: (B17010_036E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Female householder, no spouse present:!!With related children of the householder under 18 years: (B17010_037E)",Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total: (B11003_001E),Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total:!!Married-couple family: (B11003_002E),Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total:!!Married-couple family:!!With own children of the householder under 18 years: (B11003_003E),Family Type by Presence and Age of Related Children Under 18 Years_Estimate!!Total: (B11004_001E),Family Type by Presence and Age of Related Children Under 18 Years_Estimate!!Total:!!Married-couple family: (B11004_002E),Family Type by Presence and Age of Related Children Under 18 Years_Estimate!!Total:!!Married-couple family:!!With related children of the householder under 18 

# Part 2: Importing Shapefiles

(For documentation on the following code, see choropleth_maps.ipynb.)

In [3]:
gdf_counties = geopandas.read_file(
    'shapefiles/cb_2023_us_county_500k/cb_2023_us_county_500k.shp')
gdf_counties['geometry'] = gdf_counties['geometry'].simplify(tolerance = 0.005)
# Creating a column that combines county names and long state names:
# (This column can then serve as a key for an upcoming merge.)
# Note that 'STATE_NAME' is used rather than 'STUSPS' for the state component
# of this column in order to match the format used within our demographic
# dataset.
gdf_counties.insert(
    0, 'County/State', 
    gdf_counties['NAMELSAD'] + ', ' + gdf_counties['STATE_NAME'])
gdf_counties.head()

,County/State,STATEFP,COUNTYFP,COUNTYNS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,"Baldwin County, Alabama",01,003,00161527,0500000US01003,01003,Baldwin,Baldwin County,AL,Alabama,06,4117725048,1132887203,"POLYGON ((-88.02858 30.22676, -87.96685 30.235..."
1,"Houston County, Alabama",01,069,00161560,0500000US01069,01069,Houston,Houston County,AL,Alabama,06,1501742235,4795415,"POLYGON ((-85.71209 31.19727, -85.69231 31.210..."
2,"Barbour County, Alabama",01,005,00161528,0500000US01005,01005,Barbour,Barbour County,AL,Alabama,06,2292160151,50523213,"POLYGON ((-85.74803 31.61918, -85.73117 31.629..."
3,"Sumter County, Alabama",01,119,00161585,0500000US01119,01119,Sumter,Sumter County,AL,Alabama,06,2340898915,24634880,"POLYGON ((-88.42145 32.30868, -88.34043 32.991..."
4,"Miller County, Arkansas",05,091,00069166,0500000US05091,05091,Miller,Miller County,AR,Arkansas,06,1616257232,36848741,"POLYGON ((-94.04343 33.55158, -94.00037 33.564..."


Merging our Census data into our county shapefiles:

(We could also have used state and county codes for this merge; this alternative approach is shown in choropleth_maps.ipnyb.)

In [4]:
gdf_counties_and_stats = gdf_counties.merge(
    df_marriage_poverty_census_data, on = 'County/State', how = 'inner')
gdf_counties_and_stats.query("STUSPS != 'PR'", inplace = True)
gdf_counties_and_stats.head()

,County/State,STATEFP,COUNTYFP,COUNTYNS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,Sex by Age_Estimate!!Total: (B01001_001E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total: (B17010_001E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Married-couple family: (B17010_003E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Married-couple family:!!With related children of the householder under 18 years: (B17010_004E),"Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Male householder, no spouse present:!!With related children of the householder under 18 years: (B17010_011E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Female householder, no spouse present: (B17010_016E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months below poverty level:!!Other family:!!Female householder, no spouse present:!!With related children of the householder under 18 years: (B17010_017E)",Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Married-couple family: (B17010_023E),Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Married-couple family:!!With related children of the householder under 18 years: (B17010_024E),"Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Male householder, no spouse present:!!With related children of the householder under 18 years: (B17010_031E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Female householder, no spouse present: (B17010_036E)","Poverty Status in the Past 12 Months of Families by Family Type by Presence of Related Children Under 18 Years by Age of Related Children_Estimate!!Total:!!Income in the past 12 months at or above poverty level:!!Other family:!!Female householder, no spouse present:!!With related children of the householder under 18 years: (B17010_037E)",Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total: (B11003_001E),Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total:!!Married-couple family: (B11003_002E),Family Type by Presence and Age of Own Children Under 18 Years_Estimate!!Total:!!Married-couple family:!!With own children of the householder under 18 years: (B11003_003E),Family Type by Presence and Age of Related Children Under 18 Years_Estimate!!Total: (B11004_001E),Family Type by Presence and Age of Related Children Under 18 Years_Estimate!!Total:!!Married-couple family: (B11004_002E),Family Type by Presence and Age of Related Children Under 18

Confirming that, having removed Puerto Rico from the results, we have only 51 unique state codes (representing all 50 states plus DC):

In [5]:
len(gdf_counties_and_stats['STUSPS'].unique())

51

# Comparing married- and non-married household poverty rates:

We'll now create county-level choropleth maps of poverty rates for (1) married-couple households and (2) non-married households. In order to make these two maps easier to compare, we'll use the same set of bins for both. We can accomplish this by defining the bin thresholds outside of the mapping function calls, then passing them to our function as a custom bin list.

The following code creates these thresholds by adding both sets of poverty rates together, then using Pandas' quantile() function to calculate a set of percentile-based thresholds.

In [6]:
custom_percentile_threshold_list = pd.concat(
    [gdf_counties_and_stats['Married Couple Household Poverty Rate'],
    gdf_counties_and_stats['Non-Married Household Poverty Rate']]
).quantile(np.linspace(0, 1, 11)).to_list()
custom_percentile_threshold_list

[0.0,
 0.02861553248041377,
 0.040557951115933,
 0.053547192224334364,
 0.07344483699297881,
 0.11943348587952429,
 0.16662865796293214,
 0.19992865315930627,
 0.23904416600017772,
 0.29333507853403135,
 0.5946902654867257]

We can also create a linear set of thresholds by determining the minimum and maximum poverty rates for both variables, then using np.linspace() to calculate an equally-spaced group of boundaries. I found the percentile-based thresholds to result in a more colorful map, however.

In [7]:
custom_bin_min = min([gdf_counties_and_stats['Married Couple Household Poverty Rate'].min(),
    gdf_counties_and_stats['Non-Married Household Poverty Rate'].min()])
custom_bin_min

0.0

In [8]:
custom_bin_max = max([gdf_counties_and_stats['Married Couple Household Poverty Rate'].max(),
    gdf_counties_and_stats['Non-Married Household Poverty Rate'].max()])
custom_bin_max

0.5946902654867257

In [9]:
custom_linear_threshold_list = np.linspace(custom_bin_min, custom_bin_max, 11)
custom_linear_threshold_list

array([0.        , 0.05946903, 0.11893805, 0.17840708, 0.23787611,
       0.29734513, 0.35681416, 0.41628319, 0.47575221, 0.53522124,
       0.59469027])

### Creating a map of married-couple household poverty rates by county:

In [10]:
# Defining data_col_alias outside of the function call to make the latter
# a bit more readable:

data_col_alias = "Married Couple Household Poverty Rate"

map_filename = f'married_couple_household_poverty_rate_acs_2022'

m = create_map_and_screenshot(
    starting_lat = 38, starting_lon = -95, 
    html_zoom_start = 5,
    screenshot_zoom_start = 6,
    gdf = gdf_counties_and_stats, 
    data_col = 'Married Couple Household Poverty Rate', 
    boundary_name_col = 'County/State',
    data_col_alias = data_col_alias, boundary_name_alias = 'County:',
    tooltip_variable_list = [], 
         tooltip_alias_list = [],
    bin_type = 'custom', custom_threshold_list = custom_percentile_threshold_list, 
    # color_scheme = 'RdYlBu_r',
    color_scheme = 'YlOrRd',
        map_filename = map_filename,
        html_map_folder = os.getcwd()+'/maps',
        png_map_folder = os.getcwd()+'/map_screenshots')

m if display_maps == True else skipped_render_explanation

['County/State', 'Married Couple Household Poverty Rate'] ['County:', 'Married Couple Household Poverty Rate']
Generating screenshot.
Removed HTML copy of map.
['County/State', 'Married Couple Household Poverty Rate'] ['County:', 'Married Couple Household Poverty Rate']


'Skipping map output in order to allow this notebook to display on GitHub. Set display_maps to True in order to show this and other maps within the notebook.'

### Creating a map of non-married household poverty rates by county:

In [11]:
data_col_alias = "Non-Married Household Poverty Rate"

map_filename = f'non_married_couple_household_poverty_rate_acs_2022'

m = create_map_and_screenshot(
    starting_lat = 38, starting_lon = -95, 
    html_zoom_start = 5,
    screenshot_zoom_start = 6,
    gdf = gdf_counties_and_stats, 
    data_col = 'Non-Married Household Poverty Rate', 
    boundary_name_col = 'County/State',
    data_col_alias = data_col_alias, boundary_name_alias = 'County:',
    tooltip_variable_list = [], 
         tooltip_alias_list = [],
    bin_type = 'custom', custom_threshold_list = custom_percentile_threshold_list,  
    color_scheme = 'YlOrRd',
    #color_scheme = 'RdYlBu_r',
        map_filename = map_filename,
        html_map_folder = os.getcwd()+'/maps',
        png_map_folder = os.getcwd()+'/map_screenshots')

m if display_maps == True else skipped_render_explanation

['County/State', 'Non-Married Household Poverty Rate'] ['County:', 'Non-Married Household Poverty Rate']
Generating screenshot.
Removed HTML copy of map.
['County/State', 'Non-Married Household Poverty Rate'] ['County:', 'Non-Married Household Poverty Rate']


'Skipping map output in order to allow this notebook to display on GitHub. Set display_maps to True in order to show this and other maps within the notebook.'

## Mapping the difference between these two poverty rates:

In [12]:
data_col_alias = "Non-Married/Married Household Poverty Rate Difference"

map_filename = f'non_married_married_poverty_rate_difference_acs_2022'

m = create_map_and_screenshot(
    starting_lat = 38, starting_lon = -95, 
    html_zoom_start = 5,
    screenshot_zoom_start = 6,
    gdf = gdf_counties_and_stats, 
    data_col = 'Non-Married/Married Household Poverty Rate Difference', 
    boundary_name_col = 'County/State',
    data_col_alias = data_col_alias, boundary_name_alias = 'County:',
    tooltip_variable_list = [], 
         tooltip_alias_list = [],
    bin_type = 'percentile',  
    color_scheme = 'Blues',
        map_filename = map_filename,
        html_map_folder = os.getcwd()+'/maps',
        png_map_folder = os.getcwd()+'/map_screenshots')

m if display_maps == True else skipped_render_explanation

['County/State', 'Non-Married/Married Household Poverty Rate Difference'] ['County:', 'Non-Married/Married Household Poverty Rate Difference']
Generating screenshot.
Removed HTML copy of map.
['County/State', 'Non-Married/Married Household Poverty Rate Difference'] ['County:', 'Non-Married/Married Household Poverty Rate Difference']


'Skipping map output in order to allow this notebook to display on GitHub. Set display_maps to True in order to show this and other maps within the notebook.'